Apartir de aqui comienza el resultado final

In [1]:

#Apartir de aqui comienza el resultado final
import pandas as pd
#Cargamos el dataframe en formato pickle
df = pd.read_pickle('movies_dataset_modificado.pickle')
df["name_Genres"]
type(df.at[1,"name_Genres"])

list

In [2]:
#Definimos las columnas relevantes para usar en el modelo
data = df[['title', 'name_Genres','vote_average']]
data

,title,name_Genres,vote_average
0,Toy Story,"[Animation, Comedy, Family]",7.7
1,Jumanji,"[Adventure, Fantasy, Family]",6.9
2,Grumpier Old Men,"[Romance, Comedy]",6.5
3,Waiting to Exhale,"[Comedy, Drama, Romance]",6.1
4,Father of the Bride Part II,[Comedy],5.7
...,...,...,...
45461,Subdue,"[Drama, Family]",4.0
45462,Century of Birthing,[Drama],9.0
45463,Betrayal,"[Action, Drama, Thriller]",3.8
45464,Satan Triumphant,[],0.0


In [3]:
#rellenamos los nullos de vote average por su prodemio 
data['vote_average'] = data['vote_average'].fillna(5.6)


data

C:\Users\tonto\AppData\Local\Temp\ipykernel_15324\1204070717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vote_average'] = data['vote_average'].fillna(5.6)


,title,name_Genres,vote_average
0,Toy Story,"[Animation, Comedy, Family]",7.7
1,Jumanji,"[Adventure, Fantasy, Family]",6.9
2,Grumpier Old Men,"[Romance, Comedy]",6.5
3,Waiting to Exhale,"[Comedy, Drama, Romance]",6.1
4,Father of the Bride Part II,[Comedy],5.7
...,...,...,...
45461,Subdue,"[Drama, Family]",4.0
45462,Century of Birthing,[Drama],9.0
45463,Betrayal,"[Action, Drama, Thriller]",3.8
45464,Satan Triumphant,[],0.0


In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Preprocesamiento de datos variables categoricas codificadas

mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(data.pop('name_Genres')),
                          columns=mlb.classes_, 
                          index=data.index))


In [5]:
data 

,title,vote_average,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,Toy Story,7.7,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,6.9,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,6.5,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,6.1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,5.7,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,Subdue,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45462,Century of Birthing,9.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45463,Betrayal,3.8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
45464,Satan Triumphant,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split
sampled_data = data.sample(20000, random_state=42)

# Separar los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(sampled_data, test_size=0.20, random_state=42)

In [7]:
from sklearn.neighbors import NearestNeighbors

# Crear una matriz de similitud entre películas
model = NearestNeighbors(n_neighbors=5, metric='euclidean')
model.fit(train_data.drop(['title'], axis=1))

NearestNeighbors(metric='euclidean')

In [8]:
import numpy as np

def get_recommendations( movie_title):
    # Obtener el índice de la película en el conjunto de datos
    idx = data[data['title'] == movie_title].index[0]
    # Obtener las películas más similares
    distances, indices = model.kneighbors(data.iloc[idx, 1:].values.reshape(1, -1), n_neighbors=6)
    # Obtener los títulos de las películas recomendadas
    recommended_movies = [data.iloc[indices[0][i+1]]['title'] for i in range(5)]
    return recommended_movies


In [9]:

# Ejemplo de recomendación
get_recommendations("Toy Story")


c:\Users\tonto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Brain Damage',
 "Hart's War",
 'Man on the Flying Trapeze',
 'Uncertainty',
 'Black Legion']

In [13]:
pelicula_entrada = df.loc[df['title'] ==   'Uncertainty']
generos_entrada = pelicula_entrada['name_Genres'].values[0]

generos_entrada

['Drama', 'Thriller', 'Romance']

In [28]:
train_data

,title,vote_average,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
36930,Sunny in the Dark,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41809,Assassin's Creed,5.4,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7326,Cat-Women of the Moon,3.3,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8483,Abbott and Costello Meet Dr. Jekyll and Mr. Hyde,6.7,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
20136,All Superheroes Must Die,4.3,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,Streamers,6.8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44732,Farmer Al Falfa Sees New York,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38158,A Cry in the Wilderness,0.0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
860,Talk of Angels,6.8,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [763]:
df

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,name_production_companies,id_production_companies,name_production_countries,iso_3166_1_production_countries,name_spoken_languages,iso_639_1_spoken_languages,release_year,release_month,release _day,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,[Pixar Animation Studios],[3],[United States of America],[US],[English],[en],1995.0,Octubre,Lunes,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US],"[English, Français]","[en, fr]",1995.0,Diciembre,Viernes,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US],[English],[en],1995.0,Diciembre,Viernes,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[Twentieth Century Fox Film Corporation],[306],[United States of America],[US],[English],[en],1995.0,Diciembre,Viernes,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US],[English],[en],1995.0,Febrero,Viernes,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0.0,439050,fa,Rising and falling between a man and woman.,0.072051,NaT,0.0,90.0,Released,Rising and falling between a man and woman,...,[],[],[Iran],[IR],[فارسی],[fa],NaN,NaN,NaN,0.000000
45462,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,...,[Sine Olivia],[19653],[Philippines],[PH],[],[tl],2011.0,Noviembre,Jueves,0.000000
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,[American World Pictures],[6165],[United States of America],[US],[English],[en],2003.0,Agosto,Viernes,0.000000
45464,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,...,[Yermoliev],[88753],[Russia],[RU],[],[],1917.0,Octubre,Domingo,0.000000


In [3]:
data.columns

Index(['title', 'vote_average', 'Action', 'Adventure', 'Animation', 'Aniplex',
       'BROSTA TV', 'Carousel Productions', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'],
      dtype='object')

In [79]:
#evaluamos el modelo
from sklearn.model_selection import train_test_split
sampled_data = data.sample(20000, random_state=42)

# Separar los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(sampled_data, test_size=0.20, random_state=42)

In [80]:
from sklearn.neighbors import NearestNeighbors

# Crear una matriz de similitud entre películas
model = NearestNeighbors(n_neighbors=5, metric='euclidean')
model.fit(train_data.drop(['title'], axis=1))

NearestNeighbors(metric='euclidean')

In [81]:
# Puntos de consulta en el conjunto de prueba

query_points = test_data.drop(['title'], axis=1)

# Obtener los vecinos más cercanos en el conjunto de prueba
distances, indices = model.kneighbors(query_points)

# Obtener las columnas correspondientes a los índices
train_neighbors = train_data.drop(['title'], axis=1).iloc[indices.flatten()]


from sklearn.metrics.pairwise import euclidean_distances

# Calcular la distancia euclidiana entre los vecinos encontrados y los puntos de prueba
# Obtener las columnas correspondientes a los índices
distances_euclidean = euclidean_distances(query_points, train_neighbors)

# Obtener el promedio de la distancia euclidiana para evaluar el rendimiento del modelo
average_distance = np.mean(distances_euclidean)
print("Distancia euclidiana promedio:", average_distance)


Distancia euclidiana promedio: 2.8321065046605605


In [87]:

#evaluamos el modelo si no tuviera la columna vote_average
from sklearn.model_selection import train_test_split

data_og= data.drop(["vote_average"], axis=1)
sampled_data = data_og.sample(20000, random_state=42)

# Separar los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(sampled_data, test_size=0.20, random_state=42)

In [88]:
from sklearn.neighbors import NearestNeighbors

# Crear una matriz de similitud entre películas
model = NearestNeighbors(n_neighbors=5, metric='euclidean')
model.fit(train_data.drop(['title'], axis=1))

NearestNeighbors(metric='euclidean')

In [89]:
# Puntos de consulta en el conjunto de prueba

query_points = test_data.drop(['title'], axis=1)

# Obtener los vecinos más cercanos en el conjunto de prueba
distances, indices = model.kneighbors(query_points)

# Obtener las columnas correspondientes a los índices
train_neighbors = train_data.drop(['title'], axis=1).iloc[indices.flatten()]


from sklearn.metrics.pairwise import euclidean_distances

# Calcular la distancia euclidiana entre los vecinos encontrados y los puntos de prueba
# Obtener las columnas correspondientes a los índices
distances_euclidean = euclidean_distances(query_points, train_neighbors)

# Obtener el promedio de la distancia euclidiana para evaluar el rendimiento del modelo
average_distance = np.mean(distances_euclidean)
print("Distancia euclidiana promedio:", average_distance)

Distancia euclidiana promedio: 1.697275550664217


# Si estás utilizando la distancia euclidiana como medida de similitud, un valor más bajo de distancia euclidiana promedio indica una mayor similitud entre los puntos de prueba y sus vecinos más cercanos.  El modelo sin vote_average tiene menor distancia por lo que deberia dar como resultado titulos con mas similitud

In [92]:
import numpy as np

def get_recommendations( movie_title):
    # Obtener el índice de la película en el conjunto de datos
    idx = data_og[data_og['title'] == movie_title].index[0]
    # Obtener las películas más similares
    distances, indices = model.kneighbors(data_og.iloc[idx, 1:].values.reshape(1, -1), n_neighbors=6)
    # Obtener los títulos de las películas recomendadas
    recommended_movies = [data_og.iloc[indices[0][i+1]]['title'] for i in range(5)]
    return recommended_movies

In [111]:
# Ejemplo de recomendación
get_recommendations("Akira")

c:\Users\tonto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['American Me',
 'Annapolis',
 'Carnosaur 3: Primal Species',
 'The Mangler',
 'Merry Madagascar']

In [108]:
pelicula_entrada = df.loc[df['title'] ==   'Merry Madagascar']
generos_entrada = pelicula_entrada['name_Genres'].values[0]

generos_entrada

['Family', 'Animation']

In [109]:
#Exportamos la tabla para el modelo
data_og.to_csv('Data_ML.csv', index=False)